<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/pytorch/blob/main/custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np

In [2]:
!git clone https://github.com/emanhamed/Houses-dataset

Cloning into 'Houses-dataset'...
remote: Enumerating objects: 2166, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 2166 (delta 0), reused 0 (delta 0), pack-reused 2165
Receiving objects: 100% (2166/2166), 176.26 MiB | 35.91 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [3]:
df=pd.read_csv("Houses-dataset/Houses Dataset/HousesInfo.txt",header=None,delim_whitespace=True,
               names=["bedrooms","bathrooms","size","zipcode","price"])

In [ ]:
def cleanData(df):
    # compute the number of entries per zipcode
    zipcodes=df['zipcode'].value_counts().keys().tolist()
    counts=df['zipcode'].value_counts().tolist()
    #discard all zipcodes ocurring less than 20 times
    for count,zipcode in zip(counts,zipcodes):
      if count<20:
        idx=df[df['zipcode']==zipcode].index
        df.drop(idx,inplace=True)
    return df

In [ ]:
dataset=cleanData(df)

In [ ]:
import shutil
from_prefix="Houses-dataset/Houses Dataset/"
to_prefix="images/"
suffix="_frontal.jpg"
!mkdir -p images
for newidx,oldidx in enumerate(df.index.tolist()):
  oldname=from_prefix+str(oldidx)+suffix
  newname=to_prefix+str(newidx)+suffix
  #print("moving from %s to %s"%(oldname,newname))
  shutil.copy(oldname,newname)

In [ ]:
dataset.to_csv("cleansedDataset.csv",index=False)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,csvFile,imgDir):
    dataset=pd.read_csv(csvFile)
    dummy=pd.get_dummies(dataset['zipcode'])
    price=dataset['price']
    df=dataset.drop(['price','zipcode'],axis=1)
    self.data=pd.concat([dataset,dummy,price],axis=1)
  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    #
    return self.data.iloc[idx,:-1].to_numpy(),self.data.iloc[idx,-1]

In [ ]:
dataset=CustomDataset("cleansedDataset.csv","images")

In [ ]:
itr=iter(dataset)
x=next(itr)


In [ ]:
x

(array([5.0000e+00, 3.0000e+00, 2.5200e+03, 9.3446e+04, 7.8900e+05,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00]), 789000)

In [ ]:
train_loader=DataLoader(dataset,batch_size=32,shuffle=True)

In [ ]:
itr=iter(train_loader)

In [ ]:
x=next(itr)

In [ ]:
x[0].size()

torch.Size([32, 13])

In [ ]:
dummy=pd.get_dummies(dataset['zipcode'])
price=dataset['price']
df=dataset.drop(['price','zipcode'],axis=1)
result=pd.concat([dataset,dummy,price],axis=1)
result

,bedrooms,bathrooms,size,zipcode,price,91901,92276,92677,92880,93446,93510,94501,94531,price
30,5,3.0,2520,93446,789000,0,0,0,0,1,0,0,0,789000
32,3,2.0,1802,93446,365000,0,0,0,0,1,0,0,0,365000
39,3,3.0,2146,93446,455000,0,0,0,0,1,0,0,0,455000
80,4,2.5,2464,91901,599000,1,0,0,0,0,0,0,0,599000
81,2,2.0,1845,91901,529800,1,0,0,0,0,0,0,0,529800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,5,2.0,2066,94531,399900,0,0,0,0,0,0,0,1,399900
531,4,3.5,9536,94531,460000,0,0,0,0,0,0,0,1,460000
532,3,2.0,2014,94531,407000,0,0,0,0,0,0,0,1,407000
533,4,3.0,2312,94531,419000,0,0,0,0,0,0,0,1,419000
